In [1]:
department_id = 2
sheet_name = '2 DMDG -3 not send'

In [6]:
# import library
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
from datetime import datetime
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1ztv0A25e7Z0egykod3pddwnxAr87VM5oyzuDGoF8M9o/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data), sheet_name=sheet_name)
df = df.dropna()
df['Phone number'] = df['Phone number'].astype(int)
df

/Users/hari/Software/virtualenv/aavin/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
<ipython-input-6-5f1e2903a8fa>:18: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(BytesIO(data), sheet_name=sheet_name)


,S No,Department,Roll number,Name,Phone number
0,1,KULT,1,Gowthaman,9003832999
1,2,KULT,2,Hari,7010349412
2,3,KULT,3,Vivek,9585488448


## Validate the Google Sheet

In [36]:
df['Phone number'] = df['Phone number'].astype(str)
df[df['Phone number'].str.len() != 10]

,S No,Department,Roll number,Name,Phone number


## Save Applicant List From Sheet

In [14]:
df['Phone number'] = df['Phone number'].astype(int)
for index, row in df.iterrows():
    person_obj = Person(name=row['Name'], 
                       phone_number=row['Phone number'],
                       department_id=department_id, 
                       roll_number=row['Roll number'])
    person_obj.save()
    print('{} added to DB.'.format(row['Name']))


Gowthaman added to DB.
Hari added to DB.
Vivek added to DB.


## Sending message Via Netfision

In [4]:
def send_message_via_netfision(mobile, message):
    payload = {'ClientId': 'c9dc2b72-a38c-4e29-9834-17fe1ef6df3f', 'ApiKey' :'0fa9908c-1e67-4ccb-86fc-2363f7f75839', 'SenderID' : 'KULTIV', 'fl':'0', 'gwid':'2', 'sid':'KULTIV'} 
    headers = {}
    url     = 'http://sms.tnvt.in/vendorsms/pushsms.aspx'
    payload['msg'] = message
    payload['msisdn'] = mobile
    res = requests.post(url, data=payload, headers=headers)
    return res

## Send Message to Applicant And save in Person Message Table

In [14]:
department_obj = Department.objects.get(id=department_id)
# ip_address = 'http://192.168.43.88:8000/bulk/message/update/person/message/?id='
ip_address = 'http://server.aavincoimbatore.com/bulk/message/update/person/message/?id='
for person in Person.objects.filter(id=4169):
    roll_number = department_obj.short_name + person.roll_number.zfill(4)
    mobile_number = person.phone_number
    name = person.name.upper()
    person_message_obj = PersonMessage(group_id=1,
                                   person=person,
                                   message='',
                                   response_status_code='88888'
                                  )
    person_message_obj.save()
    ack_link = ip_address + str(person_message_obj.id)
    
    message = f"""Dear {name}, 
The hall ticket for {department_obj.name} post at Aavin Coimbatore has been despatched to you by professional courier/post. Your roll number is {roll_number}.
    
The written exam for the same has been scheduled on SUNDAY {department_obj.session} 14-02-2021 at {department_obj.time_start}. Your exam venue is: Hindusthan College of Arts and Science, Avinashi Rd, Behind Nava India, Coimbatore - 641028.
    
Also bring along a photo ID for the exam.

Clik the below link to acknowledge 

{ack_link}

    Manager(Admin)
    Aavin Coimbatore"""
    

    person_message_obj.message = message
    person_message_obj.save()
#     netvision message
    response = send_message_via_netfision(mobile_number, message)
    response_json = response.json()
#     update message status to table
    person_message_obj.response_json = response_json
    person_message_obj.response_status_code = response_json['ErrorCode']
    person_message_obj.response_message = response_json['ErrorMessage']
    person_message_obj.save()
    print('Message send and updated  to {} - {}'.format(mobile_number, roll_number))


Message send and updated  to 7010349412 - MCVL0002


In [6]:
send_message_via_netfision(7010349412, 'Hari').json()

{'ErrorCode': '000',
 'ErrorMessage': 'Success',
 'JobId': '62177981',
 'MessageData': [{'Number': '917010349412',
   'MessageParts': [{'MsgId': '917010349412-4ef685a7a75c4e9599d3de296a06636c',
     'PartId': 1,
     'Text': 'Hari'}]}]}

In [9]:
def serve_and_print_updated_excel(department_id):
    for person_message in PersonMessage.objects.filter(person__department_id=department_id):
        print('{},{},{},{},{}'.format(person_message.person.name, person_message.person.phone_number, person_message.person.roll_number, person_message.is_acknowledged, person_message.acknowledged_at))
        
        

In [10]:
serve_and_print_updated_excel(5)

Hari,7010349412,2,Dear HARI, the hall ticket for MANAGER - CIVIL post at Aavin Coimbatore has been despatched to you by professional courier/post. Your roll number is MCVL0002.
    
The written exam for the same has been scheduled on SUNDAY Morning 14-02-2021 at 10.30 AM. Your exam venue is: Hindusthan College of Arts and Science, Avinashi Rd, Behind Nava India, Coimbatore - 641028.
    
Also bring along a photo ID for the exam.

Follow the like to acknowledge http://192.168.43.88:8000/bulk/message/update/person/message/?id=38

    Manager(Admin)
    Aavin Coimbatore,True,2021-02-09 11:07:33.761390+00:00
Hari,7010349412,2,Dear HARI, the hall ticket for MANAGER - CIVIL post at Aavin Coimbatore has been despatched to you by professional courier/post. Your roll number is MCVL0002.
    
The written exam for the same has been scheduled on SUNDAY Morning 14-02-2021 at 10.30 AM. Your exam venue is: Hindusthan College of Arts and Science, Avinashi Rd, Behind Nava India, Coimbatore - 641028.
  

In [1]:
import pandas as pd
person_message_obj = PersonMessage.objects.all().order_by('person__department', 'person__roll_number')
person_message_list = list(person_message_obj.values_list('person__name', 'person__phone_number', 'person__roll_number', 'person__department','person__department__name', 'person__department__short_name','time_created', 'response_status_code', 'message', 'is_acknowledged', 'acknowledged_at'))
person_message_column = ['Name', 'Mobile Number', 'Number', 'department_id','Department Name', 'Department Short Name', 'Message Sent Time', 'status', 'Message', 'is_ack', 'Acknowledged Time']
person_message_df = pd.DataFrame(person_message_list, columns=person_message_column)
person_message_df['Roll Number'] = person_message_df['Department Short Name'] + person_message_df['Number']
person_message_df['Message Sent Time'] = person_message_df["Message Sent Time"].dt.strftime("%m-%d-%Y %I:%M %p").astype(str)
person_message_df['Acknowledged Time'] = person_message_df["Acknowledged Time"].dt.strftime("%m-%d-%Y %I:%M %p").astype(str)
person_message_df['Acknowledged'] = 'No'
person_message_df['Message Sent Status'] = 'Failure'
person_message_df.loc[person_message_df['is_ack'] == True, 'Acknowledged'] = 'Yes'
person_message_df.loc[person_message_df['status'] == '000', 'Message Sent Status'] = 'Success'
person_message_df



/Users/hari/Software/virtualenv/aavin/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,Name,Mobile Number,Number,department_id,Department Name,Department Short Name,Message Sent Time,status,Message,is_ack,Acknowledged Time,Roll Number,Acknowledged,Message Sent Status
0,mohamed riyaaza NA,7401493434,1,1,MANAGER - CIVIL,MCVL,02-09-2021 01:49 PM,000,"Dear MOHAMED RIYAAZA NA, \nThe hall ticket for...",True,02-10-2021 04:29 AM,MCVL1,Yes,Success
1,b.aiswaraya,9500417120,10,1,MANAGER - CIVIL,MCVL,02-09-2021 01:49 PM,000,"Dear B.AISWARAYA, \nThe hall ticket for MANAGE...",False,nan,MCVL10,No,Success
2,s.aravind,7904423886,100,1,MANAGER - CIVIL,MCVL,02-09-2021 01:49 PM,000,"Dear S.ARAVIND, \nThe hall ticket for MANAGER ...",True,02-09-2021 03:30 PM,MCVL100,Yes,Success
3,k.mugilan,9790276885,101,1,MANAGER - CIVIL,MCVL,02-09-2021 01:49 PM,000,"Dear K.MUGILAN, \nThe hall ticket for MANAGER ...",True,02-11-2021 11:49 AM,MCVL101,Yes,Success
4,s.akalia,8610419465,102,1,MANAGER - CIVIL,MCVL,02-09-2021 01:49 PM,000,"Dear S.AKALIA, \nThe hall ticket for MANAGER -...",True,02-10-2021 10:57 AM,MCVL102,Yes,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804,Tamilmozhi,9566259035,132,11,DEPUTY MANAGER - DAIRY BACTERIOLOGY,DMDB,02-11-2021 12:48 PM,000,"Dear TAMILMOZHI, \nThe hall ticket for DEPUTY ...",True,02-11-2021 06:31 PM,DMDB132,Yes,Success
3805,Deepika,9344520358,133,11,DEPUTY MANAGER - DAIRY BACTERIOLOGY,DMDB,02-11-2021 12:48 PM,000,"Dear DEEPIKA, \nThe hall ticket for DEPUTY MAN...",False,nan,DMDB133,No,Success
3806,Manisha,9942617090,134,11,DEPUTY MANAGER - DAIRY BACTERIOLOGY,DMDB,02-11-2021 12:48 PM,000,"Dear MANISHA, \nThe hall ticket for DEPUTY MAN...",True,02-11-2021 03:51 PM,DMDB134,Yes,Success
3807,Sakthi Subasree,9361543998,135,11,DEPUTY MANAGER - DAIRY BACTERIOLOGY,DMDB,02-11-2021 12:48 PM,000,"Dear SAKTHI SUBASREE, \nThe hall ticket for DE...",True,02-11-2021 04:48 PM,DMDB135,Yes,Success


In [14]:
import os
excel_file_name = 'report.xlsx'
excel_file_path = os.path.join('static/media/', excel_file_name)
with pd.ExcelWriter(excel_file_path) as writer:
    for department in Department.objects.all().order_by('id'):
        person_message_df = person_message_df.fillna('')
        filtered_df = person_message_df[person_message_df['department_id'] == department.id]
        filtered_df = filtered_df.drop(columns=['Number', 'department_id', 'is_ack', 'status'])
        filtered_df.to_excel(writer, sheet_name=str(department.short_name), index=False)

In [2]:
PersonMessage.objects.filter(is_acknowledged=True).count()

2392

In [17]:
message = """Dear MOHAMMED, 
The hall ticket for MANAGER - PURCHASE AND STORE post at Aavin Coimbatore has been despatched to you by professional courier/post. Your roll number is MPS1290.
    
The written exam for the same has been scheduled on SUNDAY Evening 14-02-2021 at 02.30 PM. Your exam venue is: Hindusthan College of Arts and Science, Avinashi Rd, Behind Nava India, Coimbatore - 641028.
    
Also bring along a photo ID for the exam.

Clik the below link to acknowledge 

http://server.aavincoimbatore.com/bulk/message/update/person/message/?id=2840

    Manager(Admin)
    Aavin Coimbatore"""

# message.find('number is')
# message[message.find('number is')+8]
splited_list = message.split('Your roll number is ')[1]
splited_list.split('.')[0]

'MPS1290'

In [4]:
person_message_obj = PersonMessage.objects.all().order_by('person__department', 'person__roll_number')
person_message_list = list(person_message_obj.values_list('person__name', 'person__phone_number' , 'person__department', 'person__department__short_name', 'message',))
person_message_column = ['Name', 'Mobile Number', 'department_id','Department Short Name','Message',]
person_message_df = pd.DataFrame(person_message_list, columns=person_message_column)
person_message_df

,Name,Mobile Number,department_id,Department Short Name,Message
0,mohamed riyaaza NA,7401493434,1,MCVL,"Dear MOHAMED RIYAAZA NA, \nThe hall ticket for..."
1,b.aiswaraya,9500417120,1,MCVL,"Dear B.AISWARAYA, \nThe hall ticket for MANAGE..."
2,s.aravind,7904423886,1,MCVL,"Dear S.ARAVIND, \nThe hall ticket for MANAGER ..."
3,k.mugilan,9790276885,1,MCVL,"Dear K.MUGILAN, \nThe hall ticket for MANAGER ..."
4,s.akalia,8610419465,1,MCVL,"Dear S.AKALIA, \nThe hall ticket for MANAGER -..."
...,...,...,...,...,...
3804,Tamilmozhi,9566259035,11,DMDB,"Dear TAMILMOZHI, \nThe hall ticket for DEPUTY ..."
3805,Deepika,9344520358,11,DMDB,"Dear DEEPIKA, \nThe hall ticket for DEPUTY MAN..."
3806,Manisha,9942617090,11,DMDB,"Dear MANISHA, \nThe hall ticket for DEPUTY MAN..."
3807,Sakthi Subasree,9361543998,11,DMDB,"Dear SAKTHI SUBASREE, \nThe hall ticket for DE..."


In [8]:
person_message_df['roll_number'] = person_message_df.apply(lambda x: find_out_roll_number(x['Message']),  axis=1)
person_message_df

,Name,Mobile Number,department_id,Department Short Name,Message,roll_number
0,mohamed riyaaza NA,7401493434,1,MCVL,"Dear MOHAMED RIYAAZA NA, \nThe hall ticket for...",MCVL0001
1,b.aiswaraya,9500417120,1,MCVL,"Dear B.AISWARAYA, \nThe hall ticket for MANAGE...",MCVL0010
2,s.aravind,7904423886,1,MCVL,"Dear S.ARAVIND, \nThe hall ticket for MANAGER ...",MCVL0100
3,k.mugilan,9790276885,1,MCVL,"Dear K.MUGILAN, \nThe hall ticket for MANAGER ...",MCVL0101
4,s.akalia,8610419465,1,MCVL,"Dear S.AKALIA, \nThe hall ticket for MANAGER -...",MCVL0102
...,...,...,...,...,...,...
3804,Tamilmozhi,9566259035,11,DMDB,"Dear TAMILMOZHI, \nThe hall ticket for DEPUTY ...",DMDB0132
3805,Deepika,9344520358,11,DMDB,"Dear DEEPIKA, \nThe hall ticket for DEPUTY MAN...",DMDB0133
3806,Manisha,9942617090,11,DMDB,"Dear MANISHA, \nThe hall ticket for DEPUTY MAN...",DMDB0134
3807,Sakthi Subasree,9361543998,11,DMDB,"Dear SAKTHI SUBASREE, \nThe hall ticket for DE...",DMDB0135


In [12]:
import os
excel_file_name = 'report_with_roll_number.xlsx'
excel_file_path = os.path.join('static/media/', excel_file_name)
with pd.ExcelWriter(excel_file_path) as writer:
    for department in Department.objects.all().order_by('id'):
        person_message_df = person_message_df.fillna('')
        filtered_df = person_message_df[person_message_df['department_id'] == department.id]
        filtered_df = filtered_df.drop(columns=['Mobile Number','department_id', 'Message'])
        filtered_df.to_excel(writer, sheet_name=str(department.short_name), index=False)

In [6]:
def find_out_roll_number(message):
    splited_list = message.split('Your roll number is ')[1]
    return splited_list.split('.')[0]